In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from math import sqrt
from sklearn.metrics import mean_squared_error

In [2]:
def label_encoder(data):
    #numeric column var ise data kısmında bulundurma.
    le = preprocessing.LabelEncoder()
    columns = data.columns
    for column in columns:
        data[column] = le.fit_transform(data[column])
    return data

In [49]:
enflasyon = pd.read_csv("../input/enflasyonverisi/enflasyonlar2.csv")
enflasyon.columns = ["tarih","enflasyon"]
enflasyon = enflasyon.apply(lambda x: x.str.replace(";", " "))
enflasyon["tarih"] = pd.to_datetime(enflasyon["tarih"]).dt.strftime("%Y-%m")
enflasyon

In [13]:
data = pd.read_csv("../input/datathon2022/train.csv")
data["tarih"] = pd.to_datetime(data["tarih"]).dt.strftime("%Y-%m")
data["enflasyon"] = None
data

In [14]:
dolar = [2.9136,2.9177,2.9561,2.9657,2.8170,2.7949,2.9495,2.8777,2.9934,2.9580,3.0027,
        3.0938,3.4370,3.5254,3.7732,3.6474,3.6347,3.5520,3.5278,3.5206,3.5203,3.4515,3.5639,3.7917,3.9190,
        3.7857,3.7555,3.7999,3.9542,4.0610,4.5245,4.5895,4.9132,6.5381,6.0580,5.5834,5.2128,5.2896,
        5.1635,5.3400,5.5705,5.9659,5.8399,5.7920,5.5813,5.8316,5.6494,5.7136,5.7465,5.9487,
        5.9819,6.2432,6.6149,6.9866,6.8221,6.8534,6.9745,7.3513,7.7187,8.3467,7.8328,7.4403,7.3151,
        7.4269,8.2487,8.2891,8.4894,8.7087,8.4539,8.3164,8.8909,9.6070]

In [15]:
test = pd.read_csv("../input/datathon2022/testFeatures.csv")
test = test.set_index("id")
all_dates_df = pd.concat([data,test], axis = 0)
all_dates_df["tarih"] = pd.to_datetime(all_dates_df["tarih"]).dt.strftime("%Y-%m")
all_dates = all_dates_df["tarih"].unique()
dolar_df = pd.DataFrame(columns = ["tarih","dolar"])
dolar_df["tarih"] = all_dates
dolar_df["dolar"] = dolar
dolar_df

In [16]:
new_data = pd.DataFrame()
for date in data["tarih"].unique():
    temp_df = data.loc[data["tarih"] == date]
    temp_df2 = enflasyon.loc[enflasyon["tarih"] == date]
    temp_df3 = dolar_df.loc[dolar_df["tarih"] == date]
    enf_value = temp_df2["enflasyon"].values[0]
    dolar_value = temp_df3["dolar"].values[0]
    temp_df["enflasyon"] = enf_value
    temp_df["dolar"] = dolar_value
    new_data = new_data.append(temp_df)
new_data

In [19]:
new_data["ürün besin değeri"] = new_data["ürün besin değeri"].astype(int)
new_data["ürün fiyatı"] = new_data["ürün fiyatı"].astype(float)
new_data["enflasyon"] = new_data["enflasyon"].astype(float)
new_data["dolar"] = new_data["dolar"].astype(float)
new_data

In [20]:
tarih = new_data.iloc[:,0:1].values
ürün = new_data.iloc[:,1:2].values
ürün_besin_degeri = new_data.iloc[:,2:3].values
ürün_kategorisi = new_data.iloc[:,3:4].values
ürün_fiyatı = new_data.iloc[:,4:5].values
ürün_üretim_yeri = new_data.iloc[:,5:6].values
market = new_data.iloc[:,6:7].values
sehir = new_data.iloc[:7:8].values
enflasyon = new_data.iloc[:,8:9].values
dolar = new_data.iloc[:,9:10].values
dolar

In [ ]:
# tarih, ürün, ürün kategorisi, ürün üretim yeri, market, şehir label encoding

In [26]:
values_of_label_encoding_data = new_data[["tarih","ürün","ürün kategorisi","ürün üretim yeri","market","şehir"]]
values_of_label_encoding_data

In [27]:
label_encoding_data = label_encoder(values_of_label_encoding_data)
label_encoding_data

In [30]:
final_data = pd.concat([label_encoding_data,new_data[["ürün besin değeri","enflasyon","dolar","ürün fiyatı"]]],axis = 1)
final_data

In [31]:
x = final_data.drop("ürün fiyatı",axis = 1)
y = final_data["ürün fiyatı"]
X = x.values
Y = y.values

In [38]:
poly_reg = PolynomialFeatures(degree = 4) # verilen dereceye göre bir polynomial bir model oluşturur. 2.dereceden grafik gibi düşün mesela. 
x_poly = poly_reg.fit_transform(X)
# print(x_poly) # x^^0, x^^1, x^^2, x^^3 şeklinde devam ediyor.
lin_reg2 = LinearRegression()
lin_reg2.fit(x_poly,y)
y_pred = lin_reg2.predict(poly_reg.fit_transform(X))
y_pred

In [40]:
print(f"Polinomal, mean_squared_error = {sqrt(mean_squared_error(y_pred, Y))}") 

In [39]:
plt.plot(Y,color = "red")
plt.plot(y_pred,color = "blue")
plt.show()

In [43]:
from xgboost import XGBRegressor
xgb = XGBRegressor()
model_xgb = xgb.fit(X,Y)
y_pred = model_xgb.predict(X)
# import statsmodels.api as sm
# model = sm.OLS(y_pred,X).fit() 
# print(model.summary())  
print(f"XGBRegressor, mean_squared_error = {sqrt(mean_squared_error(y_pred, Y))}") 

In [44]:
plt.plot(Y,color = "red")
plt.plot(y_pred,color = "blue")
plt.show()

In [61]:
test = pd.read_csv("../input/datathon2022/testFeatures.csv")
test = test.set_index("id")
test["tarih"] = pd.to_datetime(test["tarih"]).dt.strftime("%Y-%m")
test

In [62]:
new_test = pd.DataFrame()
for date in test["tarih"].unique():
    temp_df = test.loc[test["tarih"] == date]
    temp_df2 = enflasyon.loc[enflasyon["tarih"] == date]
    temp_df3 = dolar_df.loc[dolar_df["tarih"] == date]
    enf_value = temp_df2["enflasyon"].values[0]
    dolar_value = temp_df3["dolar"].values[0]
    temp_df["enflasyon"] = enf_value
    temp_df["dolar"] = dolar_value
    new_test = new_test.append(temp_df)
new_test

In [63]:
new_test["enflasyon"] = new_test["enflasyon"].astype(float)
new_test["dolar"] = new_test["dolar"].astype(float)
new_test["ürün besin değeri"] = new_test["ürün besin değeri"].astype(int)

In [64]:
new_test["tarih"] = new_test["tarih"].replace("2021-01",60)
new_test["tarih"] = new_test["tarih"].replace("2021-02",61)
new_test["tarih"] = new_test["tarih"].replace("2021-03",62)
new_test["tarih"] = new_test["tarih"].replace("2021-04",63)
new_test["tarih"] = new_test["tarih"].replace("2021-05",64)
new_test["tarih"] = new_test["tarih"].replace("2021-06",65)
new_test["tarih"] = new_test["tarih"].replace("2021-07",66)
new_test["tarih"] = new_test["tarih"].replace("2021-08",67)
new_test["tarih"] = new_test["tarih"].replace("2021-09",68)
new_test["tarih"] = new_test["tarih"].replace("2021-10",69)
new_test["tarih"] = new_test["tarih"].replace("2021-11",70)
new_test["tarih"] = new_test["tarih"].replace("2021-12",71)
new_test

In [65]:
values_of_label_encoding_test = new_test[["ürün","ürün kategorisi","ürün üretim yeri","market","şehir"]]
values_of_label_encoding_test

In [66]:
label_test_data = label_encoder(values_of_label_encoding_test)
label_test_data

In [ ]:
#	tarih	ürün	ürün kategorisi	ürün üretim yeri	market	şehir	ürün besin değeri	enflasyon	dolar	ürün fiyatı

In [69]:
final_test = pd.concat([label_test_data,new_test[["tarih","ürün besin değeri","enflasyon","dolar"]]],axis = 1)
final_test = final_test[["tarih","ürün","ürün kategorisi","ürün üretim yeri","market","şehir","ürün besin değeri","enflasyon","dolar"]]
final_test

In [70]:
y_pred = model_xgb.predict(final_test)
y_pred

In [71]:
result_df = pd.DataFrame(columns = ["id","ürün fiyatı"])
result_df["id"] = new_test.index.values
result_df = result_df.set_index("id")
result_df["ürün fiyatı"] = y_pred
result_df

In [72]:
result_df.to_csv("resultresult.csv")